In [1]:
import torch
from torch.nn import Linear
from torch_geometric.nn import ChebConv
import torchvision
import pygsp
import networkx as nx 
from torch_geometric.data import Data
from torch_geometric.utils import from_networkx
from pygsp.graphs.nngraphs.spherehealpix import SphereHealpix
from torch_geometric.nn import knn_graph
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.neighbors import BallTree
import networkx as nx
import yaml
from healpix_pool_unpool import Healpix
import math
from coordinateconversion import CoordinateConversions
from healpix_sampling import calNpix
import healpy as hp
import cv2

from small_encoder import SmallEncoder

torch.manual_seed(1234)

In [2]:
#gra=from_networkx(gra)
data=np.load("processed_images/00000001.npy")
data=torchvision.transforms.functional.to_tensor(data)
data.shape

torch.Size([1, 3145728, 3])

In [5]:
class chebchevconvolution(torch.nn.Module):
    def __init__(self):
        super(chebchevconvolution, self).__init__()
        self.edge_index=[]
        self.weight=[]
        self.conv_layer=[]
        self.model_parameters()
        self.create_architecture()
        
    def create_architecture(self):
        self.fnet = SmallEncoder(self)        
        #self.cnet = SmallEncoder(self)
        for i in range(self.depth):
            subdivisions = int(self.healpix_resolution_calculator(self.n_pixels)/2**i)
            Graph_pygsp = SphereHealpix(subdivisions, nest=True, k=20)
            graph_netx=Graph_pygsp.to_networkx()
            graph_pyg=from_networkx(graph_netx)
            edge_idx=graph_pyg.edge_index
            self.edge_index.append(edge_idx)
            weight=graph_pyg.weight
            self.weight.append(weight)
        
            
            
    def healpix_resolution_calculator(self,nodes):
        """Calculate the resolution of a healpix graph
        for a given number of nodes.
        Args:
            nodes (int): number of nodes in healpix sampling
        Returns:
            int: resolution for the matching healpix graph
        """ 
        resolution = int(math.sqrt(nodes / 12))
        return resolution
        
    def model_parameters(self):
        filename="config.yml"
        output_channels_convolutionlayer=[]
        with open(filename) as f:
            my_dict = yaml.safe_load(f)
            self.n_pixels=my_dict["IMAGE PARAMS"]["n_pixels"]
            self.norm_fn='batch'
            self.depth=my_dict["IMAGE PARAMS"]["depth"]
            self.conv1_input_channels=my_dict["MODEL PARAMS"]["conv1_input_channels"]
            self.conv1_output_channels=my_dict["MODEL PARAMS"]["conv1_output_channels"]
            self.conv1_kernel_size=my_dict["MODEL PARAMS"]["conv1_kernel_size"]
            self.bottleneck_layer1_input_channels=my_dict["MODEL PARAMS"]["bottleneck_layer1_input_channels"]
            self.bottleneck_layer1_kernel_size=my_dict["MODEL PARAMS"]["bottleneck_layer1_kernel_size"]
            self.bottleneck_layer1_output_channels=my_dict["MODEL PARAMS"]["bottleneck_layer1_output_channels"]
            self.bottleneck_layer2_kernel_size=my_dict["MODEL PARAMS"]["bottleneck_layer2_kernel_size"]
            self.bottleneck_layer2_output_channels=my_dict["MODEL PARAMS"]["bottleneck_layer2_output_channels"]
            self.bottleneck_layer3_kernel_size=my_dict["MODEL PARAMS"]["bottleneck_layer3_kernel_size"]
            self.conv2_input_channels=my_dict["MODEL PARAMS"]["conv2_input_channels"]
            self.conv2_kernel_size=my_dict["MODEL PARAMS"]["conv2_kernel_size"]
            self.conv2_output_channels=my_dict["MODEL PARAMS"]["conv2_output_channels"]
            self.norm_fn=my_dict["MODEL PARAMS"]["norm_fn"]
            #assert (self.depth==len(self.output_channels_convolutionlayer)),"Please check the config.yml file. The depth (under image params) and len(output channel) (under Model params) should be equal"

        

    def forward(self, x):
        h=self.fnet(x, self)
        
        #self.update_block = SmallUpdateBlock(self.args, hidden_dim=hdim)

        return h
model = chebchevconvolution()
print(model)

chebchevconvolution(
  (fnet): SmallEncoder(
    (norm1): BatchNorm(32)
    (conv1): ChebConv(3, 32, K=7, normalization=sym)
    (relu1): ReLU(inplace=True)
    (layer1): mySequential(
      (0): BottleneckBlock(
        (conv1): ChebConv(32, 8, K=1, normalization=sym)
        (conv2): ChebConv(8, 8, K=3, normalization=sym)
        (conv3): ChebConv(8, 32, K=1, normalization=sym)
        (relu): ReLU(inplace=True)
        (norm1): BatchNorm(8)
        (norm2): BatchNorm(8)
        (norm3): BatchNorm(32)
      )
      (1): BottleneckBlock(
        (conv1): ChebConv(32, 8, K=1, normalization=sym)
        (conv2): ChebConv(8, 8, K=3, normalization=sym)
        (conv3): ChebConv(8, 32, K=1, normalization=sym)
        (relu): ReLU(inplace=True)
        (norm1): BatchNorm(8)
        (norm2): BatchNorm(8)
        (norm3): BatchNorm(32)
      )
    )
    (layer2): mySequential(
      (0): BottleneckBlock(
        (conv1): ChebConv(32, 16, K=1, normalization=sym)
        (conv2): ChebConv(16, 1

In [6]:
h = model(data)

In [8]:
h=h.detach().numpy()

In [9]:
np.save("test.npy",h)

In [15]:
h.shape

(1, 49152, 3)

In [13]:
coord_convert=CoordinateConversions()

In [24]:
NSIDE=64
ipix, NPIX=calNpix(NSIDE)

Approximate resolution at NSIDE 64 is 0.92 deg


In [113]:
imgWidth=3500
imgHeight=1750
phi, theta = np.degrees(hp.pix2ang(nside=NSIDE, ipix=ipix))

In [114]:
x,y=coord_convert.sphericalToSphereMapCoords(np.deg2rad(phi),np.deg2rad(theta),imgWidth,imgHeight)
x=x.astype(int)
y=y.astype(int)

In [115]:
empty_image=np.full((imgHeight,imgWidth,3),[255,255,255], dtype=np.uint8)

In [116]:
R=np.array(h[0,ipix,0])
G=np.array(h[0,ipix,1])
B=np.array(h[0,ipix,2])
RGB=np.column_stack((R,G,B))
empty_image[y,x]=RGB[ipix]

In [117]:
cv2.imwrite("test.jpg", empty_image)

True

In [118]:
empty_image.shape

(1750, 3500, 3)

In [119]:
empty_image

array([[[255, 255, 255],
        [255, 255, 255],
        [255, 255, 255],
        ...,
        [255, 255, 255],
        [255, 255, 255],
        [255, 255, 255]],

       [[255, 255, 255],
        [255, 255, 255],
        [255, 255, 255],
        ...,
        [255, 255, 255],
        [255, 255, 255],
        [255, 255, 255]],

       [[255, 255, 255],
        [255, 255, 255],
        [255, 255, 255],
        ...,
        [255, 255, 255],
        [255, 255, 255],
        [255, 255, 255]],

       ...,

       [[255, 255, 255],
        [255, 255, 255],
        [255, 255, 255],
        ...,
        [255, 255, 255],
        [255, 255, 255],
        [255, 255, 255]],

       [[255, 255, 255],
        [255, 255, 255],
        [255, 255, 255],
        ...,
        [255, 255, 255],
        [255, 255, 255],
        [255, 255, 255]],

       [[255, 255, 255],
        [255, 255, 255],
        [255, 255, 255],
        ...,
        [255, 255, 255],
        [255, 255, 255],
        [255, 255, 255]]